<a href="https://colab.research.google.com/github/apoorva2810/CS6910/blob/main/Assignment-3/DL_Assignment_3_with_attention_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.layers import Activation
from keras.layers import Concatenate,TimeDistributed
import torch
from keras.models import Model
from keras.layers import Input, LSTM, Dense, GRU,SimpleRNN
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense, RNN,GRU
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense,BatchNormalization
from keras.layers import Flatten
import math
from keras.models import Model
from keras.layers import Input, LSTM, Dense, GRU,SimpleRNN
from keras.layers import Dropout
import random
import os
from google.colab import drive
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm
from PIL import Image
import pickle
from keras.models import Model
from keras.layers import Input
from tensorflow.keras import layers, models, Model, optimizers
import tensorflow.keras as K

from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
train="/content/gdrive/MyDrive/hi.translit.sampled.train.tsv"
test="/content/gdrive/MyDrive/hi.translit.sampled.test.tsv"
val="/content/gdrive/MyDrive/hi.translit.sampled.dev.tsv"


In [ ]:

# Vectorize the data.
input_texts = []
target_texts = []

with open(train, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: len(lines) - 1]:
    target_text, input_text,_ = line.split("\t")

    ##Appending '\t' and '\n' for every word in target_texts
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)




In [ ]:
"""Preparing set for unique imput and target characters"""
input_characters = set()
target_characters = set()

for i in range( len(input_texts)):
  for char in input_texts[i]:
    input_characters.add(char)

  for char in target_texts[i]:
    target_characters.add(char)


#Convert set to list
input_characters = list(input_characters)
target_characters = list(target_characters)

In [ ]:
"""Preparing dictionary for input characters and target characters"""
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

##Finding the length of maximum input word and maximum target word

max_encoder_seq_length=-1
for i in input_texts:
  if len(i)>max_encoder_seq_length:
    max_encoder_seq_length=len(i)

max_decoder_seq_length=-1
for i in target_texts:
  if len(i)>max_decoder_seq_length:
    max_decoder_seq_length=len(i)

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

target_token_index[" "]=65
input_token_index[" "]=26

In [ ]:
max_encoder_seq_length, max_decoder_seq_length

(20, 21)

In [ ]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens), dtype="float32"
)

In [ ]:
# For validation data
input_texts_val = []
target_texts_val = []

with open(val, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: len(lines) - 1]:
    target_text, input_text,_ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    input_text=input_text
    target_text = "\t" + target_text + "\n"
    input_texts_val.append(input_text)
    target_texts_val.append(target_text)


encoder_input_data_val = np.zeros(
    (len(input_texts_val), max_encoder_seq_length), dtype="float32"
)


for i, input_text in enumerate(input_texts_val):
    for t, char in enumerate(input_text):
        encoder_input_data_val[i, t] =  input_token_index[char]

In [ ]:
##For test data
# For validation data
input_texts_test = []
target_texts_test = []

with open(test, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: len(lines) - 1]:
    target_text, input_text,_ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    input_text=input_text
    target_text = "\t" + target_text + "\n"
    input_texts_test.append(input_text)
    target_texts_test.append(target_text)


encoder_input_data_test = np.zeros(
    (len(input_texts_test), max_encoder_seq_length), dtype="float32"
)


for i, input_text in enumerate(input_texts_test):
    for t, char in enumerate(input_text):
        encoder_input_data_test[i, t] =  input_token_index[char]

In [ ]:
##For test data
input_texts_test = []
target_texts_test = []

with open(test, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: len(lines) - 1]:
    target_text, input_text,_ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    input_text=input_text
    target_text = "\t" + target_text + "\n"
    input_texts_test.append(input_text)
    target_texts_test.append(target_text)


for i, input_text in enumerate(input_texts_test):
    for t, char in enumerate(input_text):
        encoder_input_data_test[i, t] =  input_token_index[char]

In [ ]:
reverse_input_char_index=dict()
for char, i in input_token_index.items():
  reverse_input_char_index[i]=char

reverse_target_char_index=dict()
for char, i in target_token_index.items():
  reverse_target_char_index[i]=char

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, RNN
from keras.layers import dot
from tensorflow.keras.layers import Activation
from keras.layers import concatenate
from keras.layers import Attention,TimeDistributed

In [1]:
"""Before executing the next cell download attention.py from here: https://github.com/thushv89/attention_keras/blob/322a16ee147122026b63305aaa5e899d9e5de883/src/layers/attention.py"""

'Before executing the next cell download attention.py from here: https://github.com/thushv89/attention_keras/blob/322a16ee147122026b63305aaa5e899d9e5de883/src/layers/attention.py'

In [ ]:
from google.colab import files
src = list(files.upload().values())[0]
open('attention.py','wb').write(src)

Saving attention.py to attention.py


4645

In [ ]:
from attention import AttentionLayer

In [ ]:

#### Testing of accuracy word by word check


def ValAcc2(x, y, enc_model,dec_model, beam_size):
  #onehotlist=[]
  correct_predict=0
  pred_words=[]
  temp=""
  for i, input_text in enumerate(x):

    input_seq = x[i:i+1]
    decoded_word = decode_sequence1(input_seq, enc_model, dec_model,beam_size)

    temp=decoded_word
    #print(decoded_word)
    #print("---",y[i])
    pred_words.append(decoded_word+'\n')

    true_word=y[i].split('\t')[1].split('\n')[0]
    if decoded_word==true_word:
      correct_predict=correct_predict+1

    #print("Decoded_word: ", decoded_word, "\tTrue_word: ",true_word)
  
  
  valid_acc=correct_predict/len(x)    
  return valid_acc, pred_words  #, decoded_word #,correct_predict,temp






In [ ]:
def beam_search_decoder(predictions, target_sequence_index,top_k=3):

  #initially
  output_sequences=[("",0)]

  for token_probs in predictions:
    new_sequences=[]

   
    for old_seq, old_score in output_sequences:
      for char_index in range(len(token_probs)):
        new_seq = old_seq+target_sequence_index[char_index]

        new_score= old_score + math.log(token_probs[char_index])
        #logging based of ASCII Value might be incorrect --> Clear

        new_sequences.append((new_seq, new_score))

    
    output_sequences=sorted(new_sequences, key= lambda val: val[1], reverse=True)

    #Picking first k based on scores
    output_sequences = output_sequences[:top_k]

  
  return output_sequences

In [ ]:
def decode_sequence1(input_seq, enc_model, dec_model,beam_size):
    attention_weights=[]
    enc_out,states_value = enc_model.predict(input_seq)
    out_tokens2d=[]
    

    #>>we will give 1 char at a time as we have defined in the inference model<<#
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0][0] = target_token_index['\t']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    x=0
   
    while not stop_condition:
        output_tokens, attention,h= dec_model.predict(
          [enc_out,states_value,target_seq])
        #print("predict pass")
        #print(output_tokens.shape)
        # Sample a token
        out_tokens2d.append(output_tokens)
        sampled_token_index=np.argmax(output_tokens, axis=-1)[0, 0]
        #print(sampled_token_index)
        sampled_char = reverse_target_char_index[sampled_token_index]
        #print(sampled_char)
        decoded_sentence = decoded_sentence +sampled_char

       
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length ):
            stop_condition = True
            # print(len(decoded_sentence))
            #print(sampled_char)
            # if (sampled_char=='\n'):
            #   print("n")
            # else :
            #   print("exceeded")

        #print(sampled_char)
        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        # target_seq[0,sampled_token_index] =1
        #target_seq[0,0] =sampled_token_index
        #target_seq = np.zeros((1,max_decoder_seq_length))
        #pos=pos+1
        target_seq[0][0] = target_token_index[sampled_char]
        
        
        # Update states
        states_value = [h]
        attention_weights.append((sampled_token_index, attention))
        x=x+1
        
    out_tokens2d=np.array(out_tokens2d)
    #print(out_tokens2d.shape[0])
    len_word=out_tokens2d.shape[0]
    len_allchars=out_tokens2d.shape[3]
    #print(input_seq.shape)
    out_tokens2d=out_tokens2d.reshape(len_word,len_allchars)
    #print(out_tokens2d.shape)
    #return decoded_sentence
    """calling beam search"""
    bsd_out=beam_search_decoder(out_tokens2d,reverse_target_char_index ,beam_size)
    #decoded_sentence=bsd_out[0]
    #return decoded_sentence
    ans=bsd_out[0][0].split('\n')
    
    # predWord=ans[0]
    return ans[0]


In [ ]:
"""General function for model creation and returning accuracy on test data"""

'General function for model creation and returning accuracy on test data'

In [ ]:
def model_run(layer_size, embedding_size,epochs=5, beam_size=3):

  ##Model creation
  latent_dim=layer_size
  encoder_inputs = Input(shape=(max_encoder_seq_length,))
  # x = keras.layers.Embedding(num_encoder_tokens, 32,input_length = max_encoder_seq_length)(encoder_inputs)
  x = keras.layers.Embedding(num_encoder_tokens, embedding_size, input_length = max_encoder_seq_length)(encoder_inputs)

  encoder = GRU(latent_dim, return_sequences=True,return_state=True)

  encoder_states = []
  ops = encoder(x)
  encoder_outputs, state_h = ops[0],ops[1:]
  encoder_states+=[state_h]

  decoder_inputs = Input(shape=(max_decoder_seq_length,))
  # y = keras.layers.Embedding(num_decoder_tokens, 32,input_length = max_decoder_seq_length)(decoder_inputs)
  y = keras.layers.Embedding(num_decoder_tokens, embedding_size,input_length = max_decoder_seq_length)(decoder_inputs)

  decoder_gru = GRU(latent_dim, return_sequences=True, return_state=True)
  decoder_outputs, _ = decoder_gru(y,
                                    initial_state=encoder_states[0])



  attn_layer = AttentionLayer(name='attention_layer')
  attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

  decoder_concat_input = keras.layers.Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])


  decoder_dense = Dense(num_decoder_tokens, activation='softmax')
  decoder_outputs1 = decoder_dense(decoder_concat_input)
  model = Model([encoder_inputs, decoder_inputs], decoder_outputs1)
  # model.summary()
  model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
  )
  model.fit(
      [encoder_input_data, decoder_input_data],
      decoder_target_data,
      batch_size=32,
      epochs=epochs,
      validation_split=0.2,
  )

  #Inference model
  #encoder model

  encoder_inputs_inf = Input(batch_shape=(1,max_encoder_seq_length))

  #>>Take the embedding layer from the trained model it has learned the weights during training.<<#
  x = model.layers[2](encoder_inputs_inf)
  enc_inf_states = []

  ops = encoder(x)

  encoder_outputs_inf = ops[0] 
  state_h_inf = ops[1:]
  enc_inf_states+=[state_h_inf]
  encoder_model = Model(inputs=encoder_inputs_inf, outputs=[encoder_outputs_inf,enc_inf_states])


  #decoder model

  encoder_states_inf = Input(batch_shape=(1,max_encoder_seq_length,latent_dim))

  #>>for inf we will pass 1 character at a time<<#
  decoder_inputs_inf=Input(batch_shape=(1,1))
  #>>take 2nd embedding layer from the trained model<<#
  y = model.layers[3](decoder_inputs_inf)


  dec_states = []
  dec_inf_states = []

  decoder_state_input_h = [Input(batch_shape=(1,latent_dim))]

  ops= decoder_gru(y, initial_state=decoder_state_input_h)

  decoder_outputs = ops[0]
  state_h = ops[1:]
  dec_inf_states+=[state_h]
  dec_states+=[decoder_state_input_h]

  #this is also layer from previous trained model
  attn_out, attn_states = attn_layer([encoder_states_inf, decoder_outputs])

  #attn_out, attn_states = attn_layer([encoder_states_inf, decoder_outputs],return_attention_scores=True)
  decoder_concat_input = keras.layers.Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

  #>>decoder_dense also from trained model<<#
  decoder_outputs = decoder_dense(decoder_concat_input)


  decoder_model = Model(inputs=
      [encoder_states_inf,dec_states,decoder_inputs_inf],
      outputs=[decoder_outputs,attn_states,dec_inf_states])
  
  return ValAcc2(encoder_input_data_test, target_texts_test, encoder_model, decoder_model,beam_size)








In [ ]:
"""For wandb"""

'For wandb'

In [ ]:
!pip install wandb

     |████████████████████████████████| 1.8MB 6.7MB/s 
     |████████████████████████████████| 102kB 12.4MB/s 
     |████████████████████████████████| 133kB 27.7MB/s 
     |████████████████████████████████| 174kB 27.5MB/s 
     |████████████████████████████████| 71kB 10.8MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=ea88b1b29251fb772cdefcfbe97adb5dc2effcec0fd6b59bdf0c6c4c69ff7787
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=b6d674dd1432cd8440987dbd193617bd549f088256fb5f76a46b90c00e6d23d2
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


In [ ]:
import wandb
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
sweep_config_temp={
  "name": "DL Assign-3-attention_20eps",
  "method": "random",
  'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
  "parameters": {
        "input_embedding_size": {
            "values": [32, 64, 128]
        },
        "hidden_layer_size":{
            "values":[128,  256]  
        },
        "beam_size":{
            "values":[3, 5, 10]
        },
        
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config_temp, entity="cs20m014", project="DL Assign-3-attention_20eps")

In [ ]:
hyperparameter_defaults = dict(
    input_embedding_size=64,
    hidden_layer_size=128,
    beam_size= 5,
    )

In [ ]:
def train():
  wandb.init(config=hyperparameter_defaults)
  config = wandb.config

  embed_size=config.input_embedding_size
  l_size=config.hidden_layer_size
  beam_size=config.beam_size
  epochs=20

  # accuracy = model_run(5, n_filters, multiplier,drop,fSize, activation, batchNorm)
  # val_accuracy = model_run(n_stack,layer_type,l_size,dropout,beam_size,epochs)
  val_accuracy = model_run(l_size, embed_size,epochs, beam_size)

  wandb.log({
        "Val Accuracy": val_accuracy})

In [ ]:
"""Only for resuming sweeps"""

wandb.agent("3mvk7cgs", project="DL Assign-3-attention_20eps", function=train)


In [ ]:
"""For calculating test accuracy on best model configuration"""

accuracy, predictedWords = model_run(256, 64, 20, 5)

'For calculating test accuracy on best model configuration'

In [ ]:
"""5 (d) answer : attention heatmap for 10 random images from test data"""

In [ ]:
#Writing pred_words to .txt file
with open('/content/result.txt', 'w') as writefile:
  for word in predictedWords:
    writefile.write(word)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
def plot_attention_weights(n,encoder_inputs, attention_weights, en_id2word, fr_id2word, filename=None):
   

    if len(attention_weights) == 0:
        print('Your attention weights was empty. No attention map saved to the disk. ' +
              '\nPlease check if the decoder produced  a proper translation')
        return

    mats = []
    dec_inputs = []
    
    print(np.array(attention_weights).shape)
    for dec_ind, attn in attention_weights:
        x=attn.reshape(-1)
        dec_inputs.append(dec_ind)
        #x=attn[dec_ind][1][0][0]
        temp=[]
        for i in range(2,20):
          temp.append(x[i])
        mats.append(temp)

          
    #mats=temp
    print(np.array(mats).shape)
    attention_mat = np.transpose(np.array(mats))

    fig, ax = plt.subplots(figsize=(11, 11))
    ax.imshow(attention_mat)
    nirm = '/content/Nirmala.ttf'
    hindi_font = FontProperties(fname=nirm)

    ax.set_xticks(np.arange(attention_mat.shape[1]))
    ax.set_yticks(np.arange(attention_mat.shape[0]))
    l=len(input_texts[n])
    ax.set_xticklabels([fr_id2word[inp] if inp != 0 else "<Res>"  for inp in dec_inputs], fontproperties=hindi_font)

    #ax.set_yticklabels([en_id2word[inp] if inp != 0 else "<Res>" if i>=l  else 0 for i,inp in enumerate(encoder_inputs).ravel()])
    i=0
    english=[]
    for inp in encoder_inputs.ravel():
      if i>=l:
        temp="<Res>"
      elif inp!=0:
         temp=en_id2word[inp]
      else:
        temp="a"
      i=i+1
      english.append(temp)
    ax.set_yticklabels(english)

    # c=0
    # for i in range(20):
    #   if i > len(dec_inputs):
    #     temp="<Res>"
    #     else:
    #       temp=fr_id2word[i]

    # ax.set_xticklabels([fr_id2word[inp] c=c+1 if c>len(dec_inputs) elsefor inp in dec_inputs], fontproperties=hindi_font)
    # ax.set_yticklabels([en_id2word[inp] for inp in encoder_inputs.ravel()])

    ax.tick_params(labelsize=11)
    ax.tick_params(axis='x', labelrotation=90)



In [ ]:
import random

In [ ]:
for i in range(1,10):
   n = random.randint(0,4500)
   decoded_sentence ,output_values,result_list,attn_wts= decode_sequence(n:n+1)
   plot_attention_weights(n,encoder_input_data[n:n+1], attn_wts, reverse_input_char_index, reverse_target_char_index)

In [ ]:
def upload_files():
  from google.colab import files
  uploaded = files.upload()
  for k, v in uploaded.items():
    open(k, 'wb').write(v)
  return list(uploaded.keys())

In [ ]:
img1 = mpimg.imread('/content/1.png')
img2 = mpimg.imread('/content/2.png')
img3 = mpimg.imread('/content/3.png')
img4 = mpimg.imread('/content/4.png')
img5 = mpimg.imread('/content/5.png')
img6 = mpimg.imread('/content/6.png')
img7 = mpimg.imread('/content/7.png')
img8 = mpimg.imread('/content/8.png')
img9 = mpimg.imread('/content/9.png')
img10 = mpimg.imread('/content/10.png')

# imgplot = plt.imshow(img)

In [ ]:
img=[]
img.append(img1)
img.append(img2)
img.append(img3)
img.append(img4)
img.append(img5)
img.append(img6)
img.append(img7)
img.append(img8)
img.append(img9)
img.append(img10)


In [ ]:

i=0

fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(12, 60))
for idx,ax in enumerate(axes.ravel()): 
    imgl = img[i]
    ax.imshow(imgl, cmap = plt.get_cmap('gray'))
    #ax.title.set_text('Pred='+ xpredict[i]+'  ,  Act='+yplot[i])
    i=i+1
    if i >9 :
      break
    
plt.show()

In [2]:
"For Question 6 - Connectivity"

'For Question 6 - Connectivity'

In [ ]:
def attention_decode_sequence(input_seq):
    attention_weights=[]
    result_list, output_values = [], []
    enc_out,states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0][0] = target_token_index['\t']
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, attention,h= decoder_model.predict(
          [enc_out,states_value,target_seq])
        #output = attention[0][0][0]
        #print(output)
        #print(np.array(attention).shape)
		    # output = sigmoid(output)
		    # output_values.append(output)
      

        sampled_token_index=np.argmax(output_tokens, axis=-1)[0, 0]
        sampled_char = reverse_target_char_index[sampled_token_index]
        result_list.append(sampled_char)
        decoded_sentence = decoded_sentence +sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length ):
            stop_condition = True
        target_seq = np.zeros((1, 1))
        target_seq[0][0] = target_token_index[sampled_char]
        states_value = [h]
        attention_weights.append((sampled_token_index, attention))
        #attention_weights.append((attention))

    return decoded_sentence,output_values,result_list,attention_weights


In [3]:
"""Function of print color for a given parameter"""
def prtColor(param):
	display(html_print(''.join([cstr(txt, color=clr) for txt,clr in param])))
 


In [4]:
"""Function that visualise for a current character"""
def currentStr(s, color='black'):
  if s == ' ':
    return "<text style=color:#000;padding-left:10px;background-color:{}> </text>".format(color, s)
  else:
    return "<text style=color:#000;background-color:{}>{} </text>".format(color, s)



In [22]:
def getColor(x):
	colors = ['#85c5e1', '#88c7e2', '#91cbe5', '#97cde6', '#a9d3e8'
    '#b7d6ec', '#bbdcee', '#c7e9f1', '#eff3fc', '#f9d8d8',
    '#f7e8e6', '#f9e7d8', '#f9cdb9', '#f9a8a9', '#f99f9f',
    '#f87696', '#fa5f9f', '#fb4943', '#fc3b9b', '#fe2e9e']
	
	return colors[x]

In [23]:
def visualize(attentionWts, pred):
	txt_clrs = []
	for i in range(len(attentionWts)):
		text = (pred[i], getColor(attentionWts[i]))
		txt_clrs.append(text)
	print_color(txt_clrs)

In [ ]:
#First calling attention decode sequence to get the attention weights and the prediction word
inp=1111 #This can be set by the user on their own choice
decoded_word, output_values,result_list,attention_weights = attention_decode_sequence(encoder_input_data[inp:inp+1])

In [ ]:
visualize(attention_weights, decoder_word)